In [1]:
import pandas as pd
from scipy.stats import shapiro, ttest_ind, mannwhitneyu, spearmanr
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
def is_normal_distribution(data, alpha=0.05):
    """Проверка нормальности распределения с помощью теста Шапиро-Уилка."""
    stat, p_value = shapiro(data)
    # True, если данные нормально распределены
    return p_value > alpha 

In [5]:
def test_hypothesis_1(df):
    print("Проверка гипотезы 1:") 
    
    # Разделение данных по сменам
    shift_1 = df[df['work_shift'] == 1]['stage_4_output_danger_gas']
    shift_2 = df[df['work_shift'] == 2]['stage_4_output_danger_gas']
    
    shift_1_normal = is_normal_distribution(shift_1)
    shift_2_normal = is_normal_distribution(shift_2)
    
    # Проверка гипотез
    if shift_1_normal and shift_2_normal:
        # Если оба распределения нормальные, используем Student's t-test
        t_stat, p_value = ttest_ind(shift_1, shift_2, equal_var=False)  # Welch's t-test
        if p_value < 0.05:
            print("Отклоняем нулевую гипотезу: Доля газа зависит от смены.")
        else:
            print("Не удалось отклонить нулевую гипотезу: Доля газа НЕ зависит от смены.")
    else:
        # Если хотя бы одно распределение не является нормальным, используем Mann-Whitney U test
        u_stat, p_value = mannwhitneyu(shift_1, shift_2, alternative='two-sided')
        if p_value < 0.05:
            print("Отклоняем нулевую гипотезу: Доля газа зависит от смены.")
        else:
            print("Не удалось отклонить нулевую гипотезу: Доля газа НЕ зависит от смены.")


In [9]:
def test_hypothesis_2(df):
    print("\nПроверка гипотезы 2:")
    
    # Извлечение данных по сменам
    shift_1 = df[df['work_shift'] == 1]['stage_3_output_temp_top']
    shift_2 = df[df['work_shift'] == 2]['stage_3_output_temp_top']
    
    # Проверка нормальности распределения
    shift_1_normal = is_normal_distribution(shift_1)
    shift_2_normal = is_normal_distribution(shift_2)
    
    # Всегда сравниваем средние значения
    if shift_1_normal and shift_2_normal:
        # Если оба распределения нормальные, используем Welch's t-test
        t_stat, p_value = ttest_ind(shift_1, shift_2, equal_var=False)
    else:
        # Если хотя бы одно распределение не является нормальным, используем Mann-Whitney U test
        u_stat, p_value = mannwhitneyu(shift_1, shift_2, alternative='two-sided')
    
    # Определение смены с более низким средним значением температуры
    better_shift = "Смена 1" if shift_1.mean() < shift_2.mean() else "Смена 2"
    # Заключение о значимости
    if p_value < 0.05:
        print(f"Есть статистически значимая разница: {better_shift} лучше управляет температурой.")
    else:
        print(f"Нет статистически значимой разницы между сменами, но у {better_shift} ниже средняя температура.")

In [8]:
def test_hypothesis_3(df):
    print("\nПроверка гипотезы 3:")
    """
    Проверяет, связана ли конверсия мономера на 1 этапе с количеством подаваемой воды на 2 этапе.
    """
    # Извлечение данных
    conversion = df['stage_1_output_konv_avd']  # Конверсия мономера на 1 этапе
    water_input = df['stage_2_input_water_sum']  # Количество подаваемой воды на 2 этапе
    
    # Проверка нормальности распределения
    conversion_normal = is_normal_distribution(conversion)
    water_input_normal = is_normal_distribution(water_input)
    
    # Выбор метода корреляции
    if conversion_normal and water_input_normal:
        corr, p_value = pearsonr(conversion, water_input)
    else:
        corr, p_value = spearmanr(conversion, water_input)

    # Интерпретация результатов
    if p_value < 0.05:
        print("Отклоняем нулевую гипотезу: Есть значимая связь между конверсией мономера и количеством подаваемой воды.")
    else:
        print("Не удается отклонить нулевую гипотезу: Значимой связи между конверсией мономера и количеством подаваемой воды нет.")

In [10]:
def test_hypothesis_4(df):
    print("\nПроверка гипотезы 4:")
    # Извлечение данных
    danger_gas = df['stage_4_output_danger_gas']  # Доля опасного газа
    output_product = df['stage_4_output_product']  # Количество выходного продукта

    danger_gas_normal = is_normal_distribution(danger_gas)
    output_product_normal = is_normal_distribution(output_product)
    
    if danger_gas_normal and output_product_normal:
        corr_stat, p_value = pearsonr(danger_gas, output_product)
        method = "Коэффициент корреляции Пирсона"
    else:
        corr_stat, p_value = spearmanr(danger_gas, output_product)
        method = "Коэффициент ранговой корреляции Спирмена"
    
    # Интерпретация результатов
    if p_value < 0.05:
        print(f"Есть статистически значимая связь между долей опасного газа и количеством выходного продукта.")
    else:
        print(f"Значимой связи между долей опасного газа и количеством выходного продукта не обнаружено.")

In [15]:
def test_hypothesis_5(df):
    print("\nПроверка гипотезы 5:")
    
    # Извлечение данных
    input_polymer = df['stage_4_input_polymer']  # Количество полимера на входе
    output_product = df['stage_4_output_product']  # Количество выходного продукта
    
    input_polymer_normal = is_normal_distribution(input_polymer)
    output_product_normal = is_normal_distribution(output_product)
    
    # Выбор типа корреляционного анализа
    if input_polymer_normal and output_product_normal:
        corr_stat, p_value = pearsonr(input_polymer, output_product)
    else:
        corr_stat, p_value = spearmanr(input_polymer, output_product)
        
    # Интерпретация результатов
    if p_value < 0.05:
        print(f"Результат: Есть статистически значимая связь между количеством полимера на входе и количеством выходного продукта.")
    else:
        print(f"Результат: Значимой связи между количеством полимера на входе и количеством выходного продукта не обнаружено.")

In [16]:
def test_hypothesis_6(df):
    print("\nПроверка гипотезы 6:")
    # Извлечение данных
    top_vacuum = df['stage_2_output_top_vacuum']  # Вакуум на 2 этапе
    product_output = df['stage_4_output_product']  # Продукт на 4 этапе

    top_vacuum_normal = is_normal_distribution(top_vacuum)
    product_output_normal = is_normal_distribution(product_output)
    
    # Выбор типа корреляционного анализа
    if top_vacuum_normal and product_output_normal:
        corr_stat, p_value = pearsonr(top_vacuum, product_output)
    else:
        corr_stat, p_value = spearmanr(top_vacuum, product_output)
        
    # Интерпретация результатов
    if p_value < 0.05:
        print(f"Результат: Есть статистически значимая связь между вакуумом на 2 этапе и продуктом на 4 этапе.")
    else:
        print(f"Результат: Значимой связи между вакуумом на 2 этапе и продуктом на 4 этапе не обнаружено.")

In [17]:
def test_hypothesis_7(df):
    print("\nПроверка гипотезы 7:")
    
    # Извлечение данных
    temp_top = df['stage_3_output_temp_top']  # Температура верха на 3 этапе
    dry_residue_avg = df['stage_4_output_dry_residue_avg']  # Средний остаток на 4 этапе

    temp_top_normal = is_normal_distribution(temp_top)
    dry_residue_avg_normal = is_normal_distribution(dry_residue_avg)
    
    # Выбор типа корреляционного анализа
    if temp_top_normal and dry_residue_avg_normal:
        corr_stat, p_value = pearsonr(temp_top, dry_residue_avg)
    else:
        corr_stat, p_value = spearmanr(temp_top, dry_residue_avg)
    
    # Интерпретация результатов
    if p_value < 0.05:
        print(f"Результат: Есть статистически значимая связь между температурой верха на 3 этапе и средним остатком на 4 этапе.")
    else:
        print(f"Результат: Значимой связи между температурой верха на 3 этапе и средним остатком на 4 этапе не обнаружено.")